In [1]:
import xgboost as xgb;
import pandas as pd;


In [2]:

dataset = pd.read_csv('C:\\JITHIN\\Projects\\malicious-website_detection-usingML\\urldata.csv', dtype={'url': 'category', 'label': 'category'})


In [3]:
dataset= dataset.drop(columns=['label','url']) 

In [4]:
dataset.head()

,result,url_length,hostname_length,path_length,fd_length,count-,count@,count?,count%,count.,count=,count-http,count-https,count-www,count-digits,count-letters,count_dir,use_of_ip,short_url
0,0,22,14,0,0,0,0,0,0,2,0,1,1,1,0,17,0,1,1
1,0,23,15,0,0,0,0,0,0,2,0,1,1,1,0,18,0,1,1
2,0,24,16,0,0,0,0,0,0,2,0,1,1,1,0,19,0,1,1
3,0,21,13,0,0,0,0,0,0,2,0,1,1,1,0,16,0,1,1
4,0,25,17,0,0,0,0,0,0,2,0,1,1,1,0,20,0,1,1


In [5]:

X =dataset.drop(columns=['result'])
y = dataset['result']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
y_train.head

<bound method NDFrame.head of 158333    0
407841    1
175705    0
11100     0
315448    0
         ..
259178    0
365838    1
131932    0
146867    0
121958    0
Name: result, Length: 360140, dtype: int64>

In [7]:
model = xgb.XGBClassifier(tree_method='hist')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


In [9]:
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to percentage
accuracy_percentage = accuracy * 100

print("Accuracy:", accuracy_percentage)

Accuracy: 99.7289972899729


In [10]:
import xgboost as xgb

# Assuming you have already trained your model and stored it in a variable named 'model'

# Save the trained model to a file
model.save_model('xgboost_model.model')


c:\Users\jithi\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [13:38:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [12]:
import modelbit
mb = modelbit.login()
import pandas as pd
import xgboost as xgb
from urllib.parse import urlparse


model = xgb.XGBClassifier(tree_method='hist')
model.load_model('xgboost_model.model')  


def predict_malicious_url(url):
 
    parsed_url = urlparse(url)
    url_length = len(url)
    hostname_length = len(parsed_url.hostname) if parsed_url.hostname else 0
    path_length = len(parsed_url.path)
    fd_length = url.count('/')
    count_dash = url.count('-')
    count_at = url.count('@')
    count_question = url.count('?')
    count_percent = url.count('%')
    count_dot = url.count('.')
    count_equal = url.count('=')
    count_http = url.count('http')
    count_https = url.count('https')
    count_www = url.count('www')
    count_digits = sum(c.isdigit() for c in url)
    count_letters = sum(c.isalpha() for c in url)
    count_dir = url.count('//')
    use_of_ip = 1 if parsed_url.hostname and parsed_url.hostname.replace('.', '').isdigit() else 0
    short_url = 1 if len(parsed_url.path) < 20 else 0

    input_data = pd.DataFrame({
        'url_length': [url_length],
        'hostname_length': [hostname_length],
        'path_length': [path_length],
        'fd_length': [fd_length],
        'count-': [count_dash],
        'count@': [count_at],
        'count?': [count_question],
        'count%': [count_percent],
        'count.': [count_dot],
        'count=': [count_equal],
        'count-http': [count_http],
        'count-https': [count_https],
        'count-www': [count_www],
        'count-digits': [count_digits],
        'count-letters': [count_letters],
        'count_dir': [count_dir],
        'use_of_ip': [use_of_ip],
        'short_url': [short_url],
    })

    prediction = model.predict(input_data)[0]
    return prediction


input_url = input()
prediction = predict_malicious_url(input_url)
print("Prediction for", input_url, ":", prediction)
mb.deploy(predict_malicious_url)
